In [ ]:
from games.utils.query_reader import QueryReader
import pandas as pd
import psycopg2.extras
import psycopg2
import time
import os

query_dir = os.path.join(os.path.abspath(''), 'queries')
qr = QueryReader(query_dir)
BASE_DATE = "'2021-05-01'"

def create_connection(): 
    return psycopg2.connect(
        dbname='main',
        host='main-us-e2.cmbsiiqeauby.us-east-2.rds.amazonaws.com',
        port=5432,
        user='doadmin',
        password='i39kew8n7jcat7l9'
    )

def run_query(connection, query, cursor=None):
    if cursor:
        cursor = connection.cursor(cursor_factory = psycopg2.extras.RealDictCursor)
    else:
        cursor = connection.cursor()
    try:
        start_time = time.time()
        cursor.execute(query)
        results = cursor.fetchall()
        end_time = time.time()

        if len(results[0]) == 1:
            results = [x[0] for x in results]
        else:
            results = [x for x in results]
        print(f'Time elapsed: {end_time - start_time}')
        print(f'Length of Results: {len(results)}')
        print(f'Sample Result: {results[0]}')
        return results
    except:
        print('rolling back')
        connection.rollback()
    finally:
        cursor.close()

def big_fetch(connection, query, cursor=None, fetch_size=30000):
    if cursor:
        cursor = connection.cursor('big_fetch_cursos', cursor_factory = cursor)
    else:
        cursor = connection.cursor('big_fetch_crusor')
    results = []
    start_time = time.time()
    cursor.execute(query)
    while True:
        rows = cursor.fetchmany(fetch_size)
        if not rows:
            break
        results.extend(rows)

    end_time = time.time()
    print(f'Time elapsed: {end_time - start_time}')
    print(f'Length of Results: {len(results)}')
    print(f'Sample Result: {results[0]}')
    cursor.close()
    return results


def read_into_df(connection, query) -> pd.DataFrame:
    start_time = time.time()
    df = pd.read_sql(query, connection)
    end_time = time.time()
    print(f'Time elapsed: {end_time - start_time}')
    print(f'DF Shape: {df.shape}')
    return df



conn = create_connection()

In [ ]:
raw_map_ids_query = qr.read_query('map_id.sql')
map_ids_query = raw_map_ids_query.format(date=BASE_DATE) 
map_ids = run_query(conn, map_ids_query)
str_map_ids = ','.join([str(x) for x in map_ids])

In [ ]:
raw_ck2 = qr.read_query('csgo_kills.sql')
ck2 = raw_ck2.format(date=BASE_DATE)
results = run_query(conn, ck2)

In [ ]:

raw_econ = qr.read_query('econ_rebuild.sql')
econ = raw_econ.format(date=BASE_DATE)
results = big_fetch(conn, econ)

In [ ]:
raw_econ = qr.read_query('econ_rebuild.sql')
econ = raw_econ.format(date=BASE_DATE)
results = run_query(conn, econ, cursor=psycopg2.extras.RealDictCursor)

In [ ]:
raw_econ = qr.read_query('econ_rebuild.sql')
econ = raw_econ.format(date=BASE_DATE)
results = run_query(conn, econ)

In [ ]:
raw_econ = qr.read_query('econ_rebuild.sql')
econ = raw_econ.format(date=BASE_DATE)
results = big_fetch(conn, econ, fetch_size=40000)
# 94s with itersize 30k
# 95s with 40k
# 96s with itersize 50k
# 99 s with 20k
# 102s with 10k

In [ ]:
from econ_rebuilder import run_econ_rebuild

raw_econ = qr.read_query('econ_rebuild.sql')
econ = raw_econ.format(date=BASE_DATE)
indf = read_into_df(conn, econ)



In [ ]:
df.to_csv('data/base_econ_df.csv', index=False)

In [ ]:

econ_df = run_econ_rebuild(df)
econ_df.head()

In [ ]:
econ_df.to_csv('output/econ_rebuild.csv')

In [ ]:
test_q = """
select crk.round_id,
       crk.killer_steam_id,
       crk.weapon_id
FROM historical_csgo.csgo_round_kills crk
LEFT JOIN historical_csgo.csgo_map_rounds cmr ON crk.round_id = cmr.round_id
where map_id not in (
SELECT map_id
FROM historical_csgo.csgo_match_maps
WHERE map_id NOT IN (
    SELECT DISTINCT(map_id)
    FROM historical_csgo.csgo_map_rounds cmr
    WHERE round_number != "t_score" + "ct_score"
    ORDER BY 1
)
  AND map_id NOT IN (
    SELECT DISTINCT(map_id)
    FROM historical_csgo.csgo_map_rounds cmr
    GROUP BY 1
    HAVING MAX(round_number) < 16
    ORDER BY 1
)
  AND map_id NOT IN (
    SELECT DISTINCT(map_id)
    FROM historical_csgo.csgo_match_maps cmm
    WHERE cmm.winner IS NULL
       OR cmm.winner = ''
)
  AND match_id NOT IN (
    SELECT match_id
    FROM historical_csgo.csgo_match_data cmd
    WHERE cmd.date < '2021-01-01'
)
    );
"""

results = big_fetch(conn, test_q, fetch_size=30000)

In [ ]:
results = run_query(conn, test_q)